In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Telus_Customer_details_Sep2022='''


DECLARE _end_dt_snpsht DATE DEFAULT '2022-11-30';


with ADC_customer_base as(

SELECT distinct customer_id,dealer_customer_id,dealer_name,account_type_name,customer_type_name
FROM `cio-datahub-enterprise-pr-183a.src_adc.bq_customer_account_details` 
where 
date(last_updt_ts) = _end_dt_snpsht
--and account_type_name!='Standalone'
--and customer_type_name='Customer'
and dealer_name='TELUS Communications Inc.'
QUALIFY ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY last_updt_ts DESC) = 1
--order by dealer_name,join_date,customer_id
)
,

Telus_customers as
(
select bacct_bus_bacct_num,bacct_billg_acct_id,cust_bus_cust_id,pi_prod_instnc_typ_cd,pi_prod_instnc_stat_ts,prod_instnc_ts,pi_cntrct_start_ts as contract_start_date
,pi_cntrct_end_ts as contract_end_date
from `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE DATE(prod_instnc_ts) = _end_dt_snpsht #Snapshot of the last day of the month
and pi_prod_instnc_typ_cd ='SMHM' #Serice type
and bus_prod_instnc_src_id = 1001 #BANs that are for home services
and pi_prod_instnc_stat_cd in ('A')
and  consldt_cust_typ_cd = 'R'
QUALIFY ROW_NUMBER() OVER (PARTITION BY bacct_bus_bacct_num ORDER BY pi_prod_instnc_stat_ts DESC) = 1
order by cust_bus_cust_id

)





, Telus_SMHM_Deact_Oct2022 as 


(


select distinct(CAST (CUST_ID as STRING)) as CUST_ID ,1 as Telus_Churn_Flag  

from `divgpras-pr-579355.SHS.SHS_DEACTS_OCT2022`


)

select * from ADC_customer_base as ADC
inner join Telus_customers as Telus
on ADC.dealer_customer_id=Telus.cust_bus_cust_id
--left join Telus_SMHM_Deact_Oct2022 c
--on ADC.dealer_customer_id=c.CUST_ID

'''

In [ ]:
Data_Sep2022=extract_bq_data(bq_client, sql=Telus_Customer_details_Sep2022)

In [ ]:
# Data_Sep2022['Telus_Churn_Flag'].sum()

In [ ]:
Data_Sep2022.info()

In [ ]:
Data_Sep2022['customer_id'].value_counts()

In [ ]:
Data_Sep2022['prod_instnc_ts'].value_counts()

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.ADC_SMHM_CustomerBase_NOV2022'

bq_table_instance= bq_client.load_table_from_dataframe(Data_Sep2022, Table_BQ,job_config=config)